<a href="https://colab.research.google.com/github/cRED-f/deep-learning/blob/main/ArtificialNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANN

## import libraries

In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Data Preprocessing

## import the datasets

In [20]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [21]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [22]:
print(y)

[1 0 1 ... 1 1 0]


## encoding categorical data

- since their is no missing data so we can skip 'taking care of missing data' part

### level encoding the 'Gender' column



In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[: , 2] = le.fit_transform(x[:,2])

In [24]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


### one hot encoding the 'Geography' column

- since france , spain  , germany have no order relation between them it can not be level encoding
- so we need to one Hot encoding

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [26]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Spliting the dataset into training set and test set

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### feature scaling

In [31]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# training the ANN

## initializing the ANN

In [32]:
ann = tf.keras.models.Sequential()

## Adding the input layer and the first hidden layer

In [34]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## adding the second hidden layer

In [35]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## adding the output layer

In [39]:
## if this is categorical problem then we should use softmax instead of sigmoid
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Training the ANN

## compiling the ANN

In [40]:
ann.compile(optimizer = 'adam' ,loss = 'binary_crossentropy' , metrics = ['accuracy'])

# if this is categorical problem then we will use categorical_crossentropy

## Training the ANN on the taining set

In [54]:
ann.fit(x_train,y_train, batch_size=32 ,epochs = 50)

Epoch 1/50
250/250 [==============================] - 2s 6ms/step - loss: 0.3602 - accuracy: 0.8547
Epoch 2/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3595 - accuracy: 0.8558
Epoch 3/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3585 - accuracy: 0.8554
Epoch 4/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3569 - accuracy: 0.8564
Epoch 5/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3566 - accuracy: 0.8591
Epoch 6/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3554 - accuracy: 0.8570
Epoch 7/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3551 - accuracy: 0.8575
Epoch 8/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3539 - accuracy: 0.8593
Epoch 9/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3527 - accuracy: 0.8593
Epoch 10/50
250/250 [==============================] - 0s 2ms/step - loss: 0.3529 - accuracy: 0.8622

# making the predictions and evaluating the model

## predicting the result of a single observation



```
Predict if the customer with the following information will leave the bank:
    Geography           : France = [1, 0, 0] <-- corresponds to
    Credit Score        : 600
    Gender              : Male   = [1]    <-- corresponds to
    Age                 : 40
    Tenure              : 3
    Balance             : 60000
    Number of Products  : 2
    Has Credit Card     : Yes    = [1]    <-- corresponds to
    Is Active Member    : Yes    = [1]    <-- corresponds to
    Estimated Salary    : 50000
    so should we say goodbye to the customer?
```

In [55]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))>0.5)

1/1 [==============================] - 0s 19ms/step
[[ True]]


# predicting the Test set results

In [56]:
y_pred = ann.predict(x_test)
y_pred = (y_pred>0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[1 0]
 [1 1]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]]


# Confusion Matrix

## Making the confusion matrix

In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[   2 1593]
 [   0  405]]


0.2035